# [Train_Acoustic_1] **Preprocess data for training**

In [ ]:
#Extract Data
%cd /workspace/content
# This cell will create a folder name [raw_data] in the root folder and extract your data into it

data_type = "lab + wav (NNSVS format)" # @param ["lab + wav (NNSVS format)", "csv + wav (DiffSinger format)", "ds + wav (DiffSinger format)"]

# The path to your data zip file
data_zip_path = "/workspace/diffsinger_dataset.zip" #@param {type:"string"}

# nnsvs-db-converter settings (lab + wav ONLY)
# These values can exceed the amount that's in your data to maximize the segment length or to keep the data as is_
# This option is necessary for variance's pitch training
# Turn it on as you want to train Variance first. Turn it off later for Acoustics.

estimate_midi = False #@param {type:"boolean"}

# Determine how long it will segment your data to based on silence phoneme placement (seconds)
segment_length = 15 #@param {type:"slider", min:5, max:35, step:1}

#Determine how many silence phoneme is allowed in the middle of each segment
max_silence_phoneme_amount = 1 #@param {type:"slider", min:0, max:50, step:1}

# leaving -S at 60 so max silence can be 60 seconds that exceeds the segment legnth cap idk why///
# making the segment length cap at 35 secs because any longer than that would make training goes really slow

# my ass dont remember why i made two... i think one is unnecessary extra but mehhh
all_shits = "/workspace/content/raw_data"
all_shits_not_wav_n_lab = "/workspace/content/raw_data/diffsinger_db"

import os
import csv
import json
import shutil
from pydub import AudioSegment

if os.path.exists("/workspace/content/raw_data"):
    shutil.rmtree("/workspace/content/raw_data")

if not os.path.exists(all_shits_not_wav_n_lab):
  os.makedirs(all_shits_not_wav_n_lab)

# using 'if not' bc i edited the wrong section which im also too lazy to fix it <3
if not data_type == "lab + wav (NNSVS format)":
    #changed to 7zip to support more compression types
    !7z x "$data_zip_path" -o{all_shits_not_wav_n_lab}
    for root, dirs, files in os.walk(all_shits):
        for filename in files:
            if filename.endswith(".lab"):
                file_path = os.path.join(root, filename)
                with open(file_path, "r") as file:
                    file_data = file.read()
                file_data = file_data.replace("SP", "pau")
                file_data = file_data.replace("br", "AP")
                with open(file_path, "w") as file:
                    file.write(file_data)

else:
    !7z x "$data_zip_path" -o{all_shits_not_wav_n_lab}


# for funny auto dict generator lmao
out = "/workspace/content/DiffSinger/dictionaries/custom_dict.txt"

phonemes = set()

def is_excluded(phoneme):
    return phoneme in ["pau", "AP", "SP", "sil"]

if data_type == "lab + wav (NNSVS format)":
    phoneme_folder_path = all_shits
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".lab"):
                fpath = os.path.join(root, file)
                with open(fpath, "r") as lab_file:
                    for line in lab_file:
                        line = line.strip()
                        if line:
                            phoneme = line.split()[2]
                            if not is_excluded(phoneme):
                                phonemes.add(phoneme)
elif data_type == "csv + wav (DiffSinger format)":
    phoneme_folder_path = all_shits_not_wav_n_lab
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".csv"):
                fpath = os.path.join(root, file)
                with open(fpath, "r", newline="") as csv_file:
                    csv_reader = csv.DictReader(csv_file)
                    for row in csv_reader:
                        if "ph_seq" in row:
                            ph_seq = row["ph_seq"].strip()
                            for phoneme in ph_seq.split():
                                if not is_excluded(phoneme):
                                    phonemes.add(phoneme)

else:
    phoneme_folder_path = all_shits_not_wav_n_lab
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".ds"):
                fpath = os.path.join(root, file)
                with open(fpath, "r") as json_file:
                    data = json.load(json_file)
                    for entry in data:
                        if "ph_seq" in entry:
                            ph_seq = entry["ph_seq"].strip()
                            phoneme_list = ph_seq.split()
                            for phoneme in phoneme_list:
                                if not is_excluded(phoneme):
                                    phonemes.add(phoneme)

with open(out, "w") as f:
    for phoneme in sorted(phonemes):
        f.write(phoneme + "	" + phoneme + "\n")

# for vowels and consonants.txt.... well adding luquid type for uta's script
dict_path = out
vowel_types = {"a", "i", "u", "e", "o", "N", "M", "NG"}
liquid_types = {"y", "w", "l", "r"} # r for english labels, it should be fine with jp too
vowel_data = []
consonant_data = []
liquid_data = []

with open(dict_path, "r") as f:
    for line in f:
        phoneme, _ = line.strip().split("\t")
        if phoneme[0] in vowel_types:
            vowel_data.append(phoneme)
        elif phoneme[0] in liquid_types:
            liquid_data.append(phoneme)
        else:
            consonant_data.append(phoneme)

vowel_data.sort()
liquid_data.sort()
consonant_data.sort()
directory = os.path.dirname(dict_path)

# make txt for language json file
vowel_txt_path = os.path.join(directory, "vowels.txt")
with open(vowel_txt_path, "w") as f:
    f.write(" ".join(vowel_data))
liquid_txt_path = os.path.join(directory, "liquids.txt")
with open(liquid_txt_path, "w") as f:
    f.write(" ".join(liquid_data))
consonant_txt_path = os.path.join(directory, "consonants.txt")
with open(consonant_txt_path, "w") as f:
    f.write(" ".join(consonant_data))


# here's a funny json append
with open(vowel_txt_path, "r") as f:
    vowel_data = f.read().split()
with open(liquid_txt_path, "r") as f:
    liquid_data = f.read().split()
with open(consonant_txt_path, "r") as f:
    consonant_data = f.read().split()
phones4json = {"vowels": vowel_data, "liquids": liquid_data}
with open("/workspace/content/nnsvs-db-converter/lang.sample.json", "w") as rawr:
    json.dump(phones4json, rawr, indent=4)


if data_type == "lab + wav (NNSVS format)":
    db_converter_script = "/workspace/content/nnsvs-db-converter/db_converter.py"
    for raw_folder_name in os.listdir(all_shits_not_wav_n_lab):
        raw_folder_path = os.path.join(all_shits_not_wav_n_lab, raw_folder_name)
        if os.path.isdir(raw_folder_path):
            if estimate_midi:
                !python {db_converter_script} -s {max_silence_phoneme_amount} -S 60 -l {segment_length} -m -c -L "/workspace/content/nnsvs-db-converter/lang.sample.json" --folder {raw_folder_path}
            else:
                #!python {db_converter_script} -s 2 --folder {raw_folder_path} 2> /dev/null
                !python {db_converter_script} -s {max_silence_phoneme_amount} -S 60 -l {segment_length} -L "/workspace/content/nnsvs-db-converter/lang.sample.json" --folder {raw_folder_path}

else:
    pass


if data_type == "lab + wav (NNSVS format)":
    for raw_folder_name in os.listdir(all_shits_not_wav_n_lab):
        raw_folder_path = os.path.join(all_shits_not_wav_n_lab, raw_folder_name)
        !rm -rf {raw_folder_path}/*.wav {raw_folder_path}/*.lab
        !mv {raw_folder_path}/diffsinger_db/* {raw_folder_path} 2> /dev/null
        !rm -rf {raw_folder_path}/diffsinger_db
        #!cp {raw_folder_path}/wavs/*.wav {raw_folder_path}
else:
    pass

# make it replace the first SP to AP cus it seems like people always forgot about it
for root, _, files in os.walk(all_shits_not_wav_n_lab):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            with open(file_path, "r", newline="") as input_file:
                csv_reader = csv.reader(input_file)
                data = [row for row in csv_reader]
                header = data[0]
                if "ph_seq" in header:
                    ph_seq_index = header.index("ph_seq")
                    if len(data) > 1 and len(data[1]) > ph_seq_index:
                        data[1][ph_seq_index] = data[1][ph_seq_index].replace("SP", "AP", 1)
            with open(file_path, "w", newline="") as output_file:
                csv_writer = csv.writer(output_file)
                csv_writer.writerows(data)

print("extraction complete!")
print("|")
print("|")
print("|")
print("I'm also nice enough to convert your data and also write your dict.txt lmao. You are welcome :)")

# [Train_Acoustic_2] **Edit Config for training Acoustic**

In [20]:
#@title #Edit Config for Acoustic
#@markdown ___

import re
import os
import yaml
import random #for the random test files lmaoz

%cd /workspace/content
clear_output()
#@markdown <font size="-1.5"> The training type you want to do
config_type = "acoustic" # @param ["acoustic", "variance"]
config_cap = config_type.upper()

spk_name = [folder_name for folder_name in os.listdir(all_shits_not_wav_n_lab) if os.path.isdir(os.path.join(all_shits_not_wav_n_lab, folder_name))]
# i used spk_name for something else cus i forgor now imma just copy and paste it
spk_names = [folder_name for folder_name in os.listdir(all_shits_not_wav_n_lab) if os.path.isdir(os.path.join(all_shits_not_wav_n_lab, folder_name))]
num_spk = len(spk_name)
raw_dir = []
for folder_name in spk_name:
    folder_path = os.path.join(all_shits_not_wav_n_lab, folder_name)
    raw_dir.append(folder_path)
if num_spk == 1:
    singer_type = "SINGLE-SPEAKER"
    diff_loss_type = "l2"
    f0_emb = "continuous"
    use_spk_id = False
    all_wav_files = []
    for root, dirs, files in os.walk("/workspace/content/raw_data/diffsinger_db"):
        for file in files:
            if file.endswith((".wav", ".ds")):
                full_path = os.path.join(root, file)
                all_wav_files.append(full_path)
    random.shuffle(all_wav_files)
    random_ass_wavs = all_wav_files[:3]
    random_ass_test_files = [os.path.splitext(os.path.basename(file))[0] for file in random_ass_wavs]

else:
    singer_type = "MULTI-SPEAKER"
    diff_loss_type = "l1"
    f0_emb = "discrete"
    use_spk_id = True
    folder_to_id = {folder_name: i for i, folder_name in enumerate(spk_name)}
    random_ass_test_files = []
    for folder_path in raw_dir:
        if data_type == "ds + wav (DiffSinger format)":
            audio_files = [f[:-4] for f in os.listdir(folder_path) if f.endswith(".ds")]
        else:
            audio_files = [f[:-4] for f in os.listdir(folder_path + "/wavs") if f.endswith(".wav")]
        folder_name = os.path.basename(folder_path)
        folder_id = folder_to_id.get(folder_name, -1)
        prefixed_audio_files = [f"{folder_id}:{audio_file}" for audio_file in audio_files]
        random_ass_test_files.extend(prefixed_audio_files[:3])
spk_id = []
for i, spk_name in enumerate(spk_name):
    spk_id_format = f"{i}:{spk_name}"
    spk_id.append(spk_id_format)

#@markdown <font size="-1.5"> Shallow Diffusion training for acoustic
use_shallow_diffusion = "true | gt_val" # @param ["false", "true | aux_val", "true | gt_val"]
if use_shallow_diffusion == "false":
    shallow = False
    gt_shallow = False
elif use_shallow_diffusion == "true | aux_val":
    shallow = True
    gt_shallow = False
else:
    shallow = True
    gt_shallow = True

#@markdown <font size="-1.5"> Half precision, or mixed precision can result in improved performance, achieving speedups on training (from [doc](https://lightning.ai/docs/pytorch/stable/common/trainer.html#precision))
# the reason why i dont add 64 is because colab is already dreadfully slow at 32 so yes im leaving it out
precision = "16-mixed" # @param ["32-true", "bf16-mixed", "16-mixed", "bf16", "16"]

#@markdown <font size="-1.5"> batch size setting, too low can cause bottleneck, too high can cause oom
batch_size = 12 # @param {type:"slider", min:1, max:100, step:1}


#@markdown <font size="-1.5"> Your model save path
save_dir = "/workspace/content/DiffSinger/MyModel_Acoustic" #@param {type:"string"}

binary_save_dir = save_dir + "/binary"

conf_dir = save_dir

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Option to use base model for finetuning

enable_finetuning = False # @param {type:"boolean"}


#@markdown <font size="-1.5"> Path to custom base model, leave blank to use [default](https://github.com/haru0l/diffsinger_models) models
#wtf haru i just looked at your readme"""""

base_model_path = "" # @param {type:"string"}

if enable_finetuning:
    pretrain = True
    if base_model_path:
        pretrain_ckpt = base_model_path
    else:
        pretrain_ckpt = f"/workspace/content/pretrain_models/{config_type}_pretrain.ckpt"
    finetune_strict_shapes = False
    finetune_ckpt_path = pretrain_ckpt
else:
    pretrain = False
    finetune_strict_shapes = True #default value
    finetune_ckpt_path = None #default value

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Model embeds check; Tension, Energy, Breathiness, Voicing | for both acoustic and variance
enable_embeds = True #@param {type: "boolean"}
if enable_embeds:
  tension_training = True
  energy_training = True
  breathiness_training = True
  voicing_training = True
else:
  tension_training = False
  energy_training = False
  breathiness_training = False
  voicing_training = False

pitch_training = "False"
if pitch_training == "False":
    pitch_training = False
    use_melody_encoder = False
    use_glide_embed = False
elif pitch_training == "True | Standard":
    pitch_training = True
    use_melody_encoder = False
    use_glide_embed = False
elif pitch_training == "True | Standard + Glide":
    pitch_training = True
    use_melody_encoder = False
    use_glide_embed = True
elif pitch_training == "True | MelodyEncoder":
    pitch_training = True
    use_melody_encoder = True
    use_glide_embed = False
else:
    pitch_training = True
    use_melody_encoder = True
    use_glide_embed = True

duration_training = True #@param {type: "boolean"}

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Pitch extractor algorithm

f0_ext = "rmvpe" # @param ["parselmouth", "rmvpe", "harvest"]
if f0_ext == "rmvpe":
    pe_ckpt_pth = "checkpoints/rmvpe/model.pt"
else:
    pe_ckpt_pth = None

#@markdown <font size="-1.5"> Select this if you want to use data augmentation (default pitch shift and time stretch values)
data_aug = True # Acoustic

with open("/workspace/content/DiffSinger/configs/base.yaml", "r") as config:
    mother = yaml.safe_load(config)
mother["pl_trainer_precision"] = precision
with open("/workspace/content/DiffSinger/configs/base.yaml", "w") as config:
    yaml.dump(mother, config)

if  data_type == "ds + wav (DiffSinger format)":
    prefer_ds = True
else:
    prefer_ds = False

if config_type == "acoustic":
    with open("/workspace/content/DiffSinger/configs/acoustic.yaml", "r") as config:
        bitch_ass_config = yaml.safe_load(config)
    bitch_ass_config["speakers"] = spk_names
    bitch_ass_config["test_prefixes"] = random_ass_test_files
    bitch_ass_config["raw_data_dir"] = raw_dir
    bitch_ass_config["num_spk"] = num_spk
    bitch_ass_config["use_spk_id"] = use_spk_id
    #bitch_ass_config["spk_ids"] = spk_id
    bitch_ass_config["diff_loss_type"] = diff_loss_type
    bitch_ass_config["f0_embed_type"] = f0_emb
    bitch_ass_config["binary_data_dir"] = binary_save_dir
    bitch_ass_config["dictionary"] = "dictionaries/custom_dict.txt"
    bitch_ass_config["augmentation_args"]["random_pitch_shifting"]["enabled"] = data_aug
    bitch_ass_config["augmentation_args"]["random_time_stretching"]["enabled"] = data_aug
    bitch_ass_config["use_key_shift_embed"] = data_aug
    bitch_ass_config["use_speed_embed"] = data_aug
    bitch_ass_config["max_batch_size"] = batch_size
    bitch_ass_config["pe"] = f0_ext
    bitch_ass_config["use_energy_embed"] = energy_training
    bitch_ass_config["use_breathiness_embed"] = breathiness_training
    bitch_ass_config["use_tension_embed"] = tension_training
    bitch_ass_config["use_voicing_embed"] = voicing_training

    bitch_ass_config["pe_ckpt"] = pe_ckpt_pth
    #shallow diff stuff
    bitch_ass_config["use_shallow_diffusion"] = shallow
    bitch_ass_config["shallow_diffusion_args"]["val_gt_start"] = gt_shallow
    #pretrain stuff
    bitch_ass_config["finetune_enabled"] = enable_finetuning
    bitch_ass_config["finetune_ckpt_path"] = finetune_ckpt_path
    bitch_ass_config["finetune_strict_shapes"] = finetune_strict_shapes

    with open("/workspace/content/DiffSinger/configs/acoustic.yaml", "w") as config:
        yaml.dump(bitch_ass_config, config)
else:
    with open("/workspace/content/DiffSinger/configs/variance.yaml", "r") as config:
        bitch_ass_config = yaml.safe_load(config)
    bitch_ass_config["speakers"] = spk_names
    bitch_ass_config["test_prefixes"] = random_ass_test_files
    bitch_ass_config["raw_data_dir"] = raw_dir
    bitch_ass_config["num_spk"] = num_spk
    bitch_ass_config["use_spk_id"] = use_spk_id
    bitch_ass_config["diff_loss_type"] = diff_loss_type
    bitch_ass_config["binary_data_dir"] = binary_save_dir
    bitch_ass_config["dictionary"] = "dictionaries/custom_dict.txt"
    bitch_ass_config["max_batch_size"] = batch_size
    bitch_ass_config["pe"] = f0_ext # i think variance uses it for pitch ref as ground-truth for pitch training soooo
    bitch_ass_config["pe_ckpt"] = pe_ckpt_pth #same goes to this one

    bitch_ass_config["predict_energy"] = energy_training
    bitch_ass_config["predict_breathiness"] = breathiness_training
    bitch_ass_config["predict_tension"] = tension_training
    bitch_ass_config["predict_pitch"] = pitch_training
    bitch_ass_config["predict_voicing"] = voicing_training

    bitch_ass_config["use_melody_encoder"] = use_melody_encoder
    bitch_ass_config["use_glide_embed"] = use_glide_embed
    bitch_ass_config["predict_dur"] = duration_training
    bitch_ass_config["binarization_args"]["prefer_ds"] = prefer_ds
    #pretrain stuff
    bitch_ass_config["finetune_enabled"] = enable_finetuning
    bitch_ass_config["finetune_ckpt_path"] = finetune_ckpt_path
    bitch_ass_config["finetune_strict_shapes"] = finetune_strict_shapes

    with open("/workspace/content/DiffSinger/configs/variance.yaml", "w") as config:
        yaml.dump(bitch_ass_config, config)

os.makedirs(save_dir, exist_ok=True)


with open("/workspace/content/DiffSinger/utils/hparams.py", "r") as f:
    hparams_py_read = f.read()
hparams_py_read = re.sub(r"args_work_dir\s*=\s*.*", f"args_work_dir = '{save_dir}'", hparams_py_read)
with open("/workspace/content/DiffSinger/utils/hparams.py", "w") as f:
    f.write(hparams_py_read)

with open("/workspace/content/DiffSinger/utils/training_utils.py", "r") as f:
    training_utils_stuff = f.read()
training_utils_stuff = re.sub("relative_path\s*=\s*.*", "relative_path = filepath.relative_to(Path('/workspace/content').resolve())", training_utils_stuff)
with open("/workspace/content/DiffSinger/utils/training_utils.py", "w") as f:
    f.write(training_utils_stuff)

print("config updated! see below for config's information")
print("|")
print("|")
print("|")
print(f"+++---{config_cap} {singer_type} TRAINING---+++")
print("|")
print("|")
print("|")
print("+++---user's settings---+++")
print("\n")
print(f"speaker name: {spk_names}")
print("\n")
print(f"data augmentation: {data_aug}")
print("\n")
print(f"pitch extractor: {f0_ext}")
print("\n")
print(f"binary data save directory: {binary_save_dir}")
print("\n")
print(f"your model will be saved to: {save_dir}")
print("\n")
print("==========================================================================================")
print("\n")
print("+++---other auto-defined settings---+++")
print("\n")
print(f"test files (auto selected): {random_ass_test_files}")
print("\n")
print("dictionary (auto generated): custom_dict.txt")
print("\n")
print(f"max_batch_size: {batch_size}")
print("\n")
print("==========================================================================================")
print("\n")
print("if you don't like or disagree with any of these options,")
print(f"you can go and edit the config at [/workspace/content/DiffSinger/configs/{config_type}.yaml]")


config updated! see below for config's information
|
|
|
+++---ACOUSTIC SINGLE-SPEAKER TRAINING---+++
|
|
|
+++---user's settings---+++


speaker name: ['data']


data augmentation: True


pitch extractor: rmvpe


binary data save directory: /workspace/content/DiffSinger/MyModel_Acoustic/binary


your model will be saved to: /workspace/content/DiffSinger/MyModel_Acoustic




+++---other auto-defined settings---+++


test files (auto selected): ['tiger_023_seg000', 'tiger_019_seg002', 'tiger_027_seg011']


dictionary (auto generated): custom_dict.txt


max_batch_size: 12




if you don't like or disagree with any of these options,
you can go and edit the config at [/workspace/content/DiffSinger/configs/acoustic.yaml]


# [Train_Acoustic_3] **Binarize the dataset**

In [21]:
#@markdown # Preprocess data
import os

# idk i just feel like 800 is a lil low for some people part 2
new_f0_max = 1600
og_script = "/workspace/content/DiffSinger/utils/binarizer_utils.py"
with open(og_script, 'r') as file:
    mate = file.read()
up_f0_val = mate.replace("f0_max = 800", f"f0_max = {new_f0_max}")
with open(og_script, 'w') as file:
    file.write(up_f0_val)

training_config = f"/workspace/content/DiffSinger/configs/{config_type}.yaml"
%cd /workspace/content/DiffSinger
os.environ['PYTHONPATH']='.'
!CUDA_VISIBLE_DEVICES=0 python /workspace/content/DiffSinger/scripts/binarize.py --config {training_config}

/workspace/content/DiffSinger
| Hparams chains:  ['configs/base.yaml', '/workspace/content/DiffSinger/configs/acoustic.yaml']
| Hparams: 
K_step: 400, K_step_infer: 400, accumulate_grad_batches: 1, audio_num_mel_bins: 128, audio_sample_rate: 44100, 
augmentation_args: {'fixed_pitch_shifting': {'enabled': False, 'scale': 0.5, 'targets': [-5.0, 5.0]}, 'random_pitch_shifting': {'enabled': True, 'range': [-5.0, 5.0], 'scale': 0.75}, 'random_time_stretching': {'enabled': True, 'range': [0.5, 2.0], 'scale': 0.75}}, base_config: [], binarization_args: {'num_workers': 0, 'shuffle': True}, binarizer_cls: preprocessing.acoustic_binarizer.AcousticBinarizer, binary_data_dir: /workspace/content/DiffSinger/MyModel_Acoustic/binary, 
breathiness_smooth_width: 0.12, clip_grad_norm: 1, dataloader_prefetch_factor: 2, dataset_size_key: lengths, dictionary: dictionaries/custom_dict.txt, 
diff_accelerator: ddim, diff_decoder_type: wavenet, diff_loss_type: l2, diff_speedup: 10, dilation_cycle_length: 4, 
dro

In [22]:
#@markdown #Train your model
%cd /workspace/content/DiffSinger
import re
import os
import yaml
#@markdown ___

#@markdown <font size="-1.5"> Step interval of when your model will be validate and save
save_interval = 2000 #@param {type:"slider", min:100, max:10000, step:100}

#@markdown ___

#@markdown ###**Only edit this section if you want to resume training**
resume_training = False #@param {type:"boolean"}

#@markdown <font size="-1.5"> path to the config you got from training
re_config_path = "/workspace/content/DiffSinger/MyModel_Acoustic" #@param {type:"string"}
model_dir = os.path.dirname(re_config_path)
save_dir = model_dir
if resume_training:
    with open("/workspace/content/DiffSinger/utils/hparams.py", "r") as f:
        hparams_py_read = f.read()
    hparams_py_read = re.sub(r"args_work_dir\s*=\s*.*", f"args_work_dir = '{save_dir}'", hparams_py_read)
    with open("/workspace/content/DiffSinger/utils/hparams.py", "w") as f:
        f.write(hparams_py_read)
    with open("/workspace/content/DiffSinger/utils/training_utils.py", "r") as f:
        training_utils_stuff = f.read()
    training_utils_stuff = re.sub("relative_path\s*=\s*.*", "relative_path = filepath.relative_to(Path('/workspace/content').resolve())", training_utils_stuff)
    with open("/workspace/content/DiffSinger/utils/training_utils.py", "w") as f:
        f.write(training_utils_stuff)

    config_path = re_config_path
    log_dir = save_dir

    !cp {model_dir}/dictionary.txt /workspace/content/DiffSinger/dictionaries/custom_dict.txt

else:
    config_path = training_config
    log_dir = conf_dir

with open(config_path, "r") as config:
    ehe = yaml.safe_load(config)
ehe["val_check_interval"] = save_interval
with open(config_path, "w") as config:
    yaml.dump(ehe, config)

# logs = log_dir
# %reload_ext tensorboard
# %tensorboard --logdir {logs}/lightning_logs

!python /workspace/content/DiffSinger/scripts/train.py --config {config_path} --exp_name ${save_dir} --reset

/workspace/content/DiffSinger
| Hparams chains:  ['configs/base.yaml', '/workspace/content/DiffSinger/configs/acoustic.yaml']
| Hparams: 
K_step: 400, K_step_infer: 400, accumulate_grad_batches: 1, audio_num_mel_bins: 128, audio_sample_rate: 44100, 
augmentation_args: {'fixed_pitch_shifting': {'enabled': False, 'scale': 0.5, 'targets': [-5.0, 5.0]}, 'random_pitch_shifting': {'enabled': True, 'range': [-5.0, 5.0], 'scale': 0.75}, 'random_time_stretching': {'enabled': True, 'range': [0.5, 2.0], 'scale': 0.75}}, base_config: ['configs/base.yaml'], binarization_args: {'num_workers': 0, 'shuffle': True}, binarizer_cls: preprocessing.acoustic_binarizer.AcousticBinarizer, binary_data_dir: /workspace/content/DiffSinger/MyModel_Acoustic/binary, 
breathiness_smooth_width: 0.12, clip_grad_norm: 1, dataloader_prefetch_factor: 2, dataset_size_key: lengths, dictionary: dictionaries/custom_dict.txt, 
diff_accelerator: ddim, diff_decoder_type: wavenet, diff_loss_type: l2, diff_speedup: 10, dilation_cy